## 多次元ランダムウォークデータを生成

In [ ]:
def generate_nd_random_walk(n, time_points=500, step_size=0.001, seed=42):
    import numpy as np
    import pandas as pd

    if seed is not None:
        np.random.seed(seed)

    # Initialize the data array with zeros
    data = np.zeros((time_points, n))
    # Initialize a small random value for each dimension
    data[0] = np.random.uniform(-step_size, step_size, n)

    for t in range(1, time_points):
        # Copy the previous position
        data[t] = data[t - 1]

        # Apply a small random change to each dimension for smoothness
        changes = np.random.uniform(-step_size, step_size, n)
        data[t] += changes

    # Creating the DataFrame
    column_names = ['v' + str(i) for i in range(1, n + 1)]
    return pd.DataFrame(data, columns=column_names, index=np.arange(time_points))

## 多次元データの可視化

主成分分析（PCA）、t-SNE、PaCMAP、およびtoorPIAを使用して多次元時系列データを2次元に可視化する

In [ ]:
def visualize_nd_data(time_series_data_nd):
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE
    import umap
    import pacmap
    import toorpia.utils as toorpia
    import numpy as np
    import time
    from sklearn.preprocessing import StandardScaler

    # Normalize the n-dimensional data
    scaler = StandardScaler()
    X_nd_normalized = scaler.fit_transform(time_series_data_nd.values)

    # Generate a color map
    cmap = plt.cm.viridis
    colors = cmap(np.linspace(0, 1, len(X_nd_normalized)))

    # Measure analysis time for PCA
    start_time = time.time()
    pca_nd = PCA(n_components=2)
    X_pca_nd = pca_nd.fit_transform(X_nd_normalized)
    pca_time = time.time() - start_time

    # Measure analysis time for t-SNE
    start_time = time.time()
    tsne_nd = TSNE(n_components=2, random_state=42)
    X_tsne_nd = tsne_nd.fit_transform(X_nd_normalized)
    tsne_time = time.time() - start_time

    # Measure analysis time for UMAP
    #start_time = time.time()
    #umap_model_nd = umap.UMAP(n_components=2, random_state=42)
    #X_umap_nd = umap_model_nd.fit_transform(X_nd_normalized)
    #umap_time = time.time() - start_time

    # PaCMAPの実行と時間測定
    start_time = time.time()
    pacmap_instance = pacmap.PaCMAP(n_components=2)
    X_pacmap_nd = pacmap_instance.fit_transform(X_nd_normalized)
    pacmap_time = time.time() - start_time

    # Measure analysis time for toorPIA
    start_time = time.time()
    X_toorpia_nd = toorpia.fit_transform(X_nd_normalized)
    toorpia_time = time.time() - start_time

    # Print analysis times
    print(f'PCA analysis time: {pca_time:.2f} seconds')
    print(f't-SNE analysis time: {tsne_time:.2f} seconds')
    #print(f'UMAP analysis time: {umap_time:.2f} seconds')
    print(f'PaCMAP analysis time: {pacmap_time:.2f} seconds')
    print(f'toorPIA analysis time: {toorpia_time:.2f} seconds')

    # Plotting
    fig, axes = plt.subplots(1, 4, figsize=(24, 5))

    # Function to create a line and point plot with color gradient
    def line_point_plot_with_gradient(ax, data, colors):
        for i in range(len(data) - 1):
            ax.plot(data[i:i+2, 0], data[i:i+2, 1], color=colors[i], linewidth=0.0, marker='o', markersize=1)

    # PCA plot
    line_point_plot_with_gradient(axes[0], X_pca_nd, colors)
    axes[0].set_title('PCA Projection ({}D)'.format(X_nd_normalized.shape[1]))

    # t-SNE plot
    line_point_plot_with_gradient(axes[1], X_tsne_nd, colors)
    axes[1].set_title('t-SNE Projection ({}D)'.format(X_nd_normalized.shape[1]))

    # UMAP plot
    #line_point_plot_with_gradient(axes[2], X_umap_nd, colors)
    #axes[2].set_title('UMAP Projection ({}D)'.format(X_nd_normalized.shape[1]))
    #axes[2].set_xlabel('Component 1')
    #axes[2].set_ylabel('Component 2')

    # PaCMAPプロット
    line_point_plot_with_gradient(axes[2], X_pacmap_nd, colors)
    axes[2].set_title('PaCMAP Projection ({}D)'.format(X_nd_normalized.shape[1]))

    # toorPIA plot
    line_point_plot_with_gradient(axes[3], X_toorpia_nd, colors)
    axes[3].set_title('toorPIA Map ({}D)'.format(X_nd_normalized.shape[1]))

    plt.show()


## 多次元ランダムウォークデータを複数系列生成しそれらを結合

In [ ]:
import pandas as pd
import numpy as np

# Generate the data
ndim = 100
data_points = 500
data1 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=41)
data2 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=42)
data3 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=43)
data4 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=44)
data5 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=45)
data6 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=46)
data7 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=47)
data8 = generate_nd_random_walk(n=ndim, time_points=data_points, seed=48)

# Combine the datasets
combined_data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8])

## 各種手法で解析して結果の表示

(処理の途中でwarningがでますが無視してください。)

In [ ]:
visualize_nd_data(combined_data)

## 結果の解説

上の解析は、高次元空間において原点をスタートポイントとする酔歩（ランダムウォーク）過程の時系列を8本生成し、各種次元削減ツールによってそれらがどのように2次元上で表現されるかを比較したものである。
（これは、製造プロセスなどで発生する「時系列データ」の解析能力をテストするためのものである。）

高次元空間中を進む酔歩には次の特徴がある。

- 酔歩は局所的にランダムに飛躍して（千鳥足のように）進む確率過程なので、局所構造としてノコギリ状の構造が表現できなければならない。
- 高次元空間では、原点からスタートする異なる8系列の酔歩の終点は、時間経過とともに互いに徐々に離れていく傾向にある。この状況を2次元マップ上で表現するには、8本の系列を原点を中心に放射状に配置するのが最適と言える。（なお、各系列が互いに離れていくという性質は、高次元空間では方向の自由度が大きいことから、異なる系列の酔歩が同じ方向に向かう確率が極めて小さくなることによるものである。）

** 以上の2点を両方表現できる手法は少ない **

- PCAは局所構造のみ正しく表現されている。しかし、大規模構造は、8本の系列のうち一部の系列しか区別することができていない。
- t-SNE, PaCMAPについては、局所構造と大規模構造の両方とも適切に表現できていない。また、系列が途中で切断されたり、他の系列と交差するなど、これらの手法が線状クラスタ（時系列データを含む）を正確に表現することができないことが分かる。